In [1]:
from profiling.funs import load, my_key, _save
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
def check(keywords):
    def find(x):
        return any(
            [word in x for word in keywords]
        )
    return find

In [4]:
def check_just(keywords):
    def find(x):
        return any(
            [word == x for word in keywords]
        )
    return find

In [5]:
def from_set(fz):
    return {name:value for (name,value) in fz}

# TensorFlow

In [ ]:
def no_idle(df):
    df = df[df['Operation']!='IDLE']
    
    return df

In [6]:
def store_tf(num):
    dct = load('profiling/results/data_{}.tflow'.format(num))

    def total_on(df, words):
        mask = df['Operation'].apply(check(words))
        return df[mask]['Avg. self-time (us)'].sum()
    
    def get(key):
        fsk=from_set(key)
        name = fsk['model_str']
        
        num=total_on(dct[key], fsk['model_str'].upper())
        
        fsk['total_ms'] = num
       
        return fsk
    
    big_list = filter(None, [get(key) for key in dct])
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    _save(df, 'datasets/total_{}.{}'.format(num, frame))

In [7]:
store_tf(num=1)

In [8]:
store_tf(num=2)

In [9]:
store_tf(num=3)

In [10]:
load('datasets/total_{}.{}'.format(1, 'tflow'))

,model_str,hp,numf,batch,nodes,total_ms
0,avg2d,4,128,64,3,8.522662e+05
1,max1d,4,32,64,2,2.847252e+04
2,conv1d,4,128,128,3,2.837115e+04
3,dense,32,32,256,1,1.112241e+05
4,avg2d,2,64,32,3,2.486354e+05
5,conv1d,8,128,512,3,3.509934e+04
6,norm1d,0,64,128,3,3.053829e+04
7,avg1d,2,32,128,1,4.347610e+03
8,conv1d,2,128,128,2,2.850172e+04
9,max1d,2,128,256,1,6.989053e+03


In [11]:
load('datasets/total_{}.{}'.format(2, 'tflow'))

,model_str,hp,numf,batch,nodes,total_ms
0,conv2d,8,64,256,3,4.879696e+05
1,norm2d,0,64,128,3,3.115935e+05
2,norm1d,0,32,32,3,3.186106e+04
3,norm1d,0,32,128,3,3.107545e+04
4,dense,32,16,512,2,5.841418e+04
5,avg2d,4,128,256,2,8.952033e+05
6,max2d,4,64,128,3,2.445547e+05
7,conv2d,8,128,128,1,9.449476e+05
8,avg1d,2,128,128,2,2.129414e+04
9,max1d,2,64,512,2,3.421036e+04


# PyTorch

In [12]:
ops_tor = {
    'conv1d': ['conv1d', 'MkldnnConvolutionBackward'],
    
    'conv2d':['conv2d', 'MkldnnConvolutionBackward'],
    
    'avg1d': ['avg_pool1d'],
    
    'avg2d': ['avg_pool2d'],
    
    'max1d': ['max_pool1d'],
    
    'max2d': ['max_pool2d'],
    
    'dense': ['addmm', 'AddmmBackward'],
    
    'norm1d': ['NativeBatchNormBackward', 'batch_norm', 'native_batch_norm'],
    
    'norm2d': ['NativeBatchNormBackward', 'batch_norm', 'native_batch_norm']
}

In [13]:
def df_total(dct, ops, clean):
    def get(key):
        key_dct = from_set(key)
        name = key_dct['model_str']
        
        total = clean(dct[key], ops[name])['total_ms'].sum()
        
        key_dct['total_ms'] = total
        
        return key_dct
    
    big_list = [get(key) for key in dct]
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    return df
    

In [14]:
def clean_tor(_df, words):
    df = _df.reset_index()
    df = df[['Name', 'CPU time avg (us)']]
    df['CPU time avg (us)'] = df['CPU time avg (us)']/1000
    df.columns = ['op', 'total_ms']
    df = df.sort_values(by='total_ms', ascending=False).reset_index(drop=True)
    
    mask = df['op'].apply(check_just(words))
    return df[mask]

In [15]:
def store_tor(num, ops, clean):
    dct = load('profiling/results/data_{}.torch'.format(num))

    df = df_total(dct, ops, clean)

    _save(df, 'datasets/total_{}.torch'.format(num))

In [16]:
def store_big(num, the_col):
    dct = load('profiling/results/data_{}.torch'.format(num))

    df = big_total(dct, the_col)

    _save(df, 'datasets/big_total_{}.torch'.format(num))

In [17]:
def big_total(dct, the_col):
    def get(key):
        key_dct = from_set(key)
        name = key_dct['model_str']
        
        total = dct[key][the_col].sum()/1000
        
        key_dct['total_ms'] = total
        
        return key_dct
    
    big_list = filter(None, [get(key) for key in dct])
    df = pd.DataFrame(data = big_list)
    
    df = df[['model_str', 'hp', 'numf', 'batch', 'nodes', 'total_ms']]
    
    return df

In [18]:
store_tor(num=1, ops=ops_tor, clean=clean_tor)

In [19]:
store_tor(num=2, ops=ops_tor, clean=clean_tor)

In [20]:
store_tor(num=3, ops=ops_tor, clean=clean_tor)

In [21]:
load('datasets/total_{}.{}'.format(1, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,max1d,4,32,64,2,0.487869
1,max1d,4,64,64,1,0.141011
2,conv1d,4,128,128,3,2.329512
3,conv2d,2,64,32,1,3.083430
4,avg2d,2,64,32,3,1.773616
5,conv1d,8,128,512,3,3.925985
6,norm1d,0,64,128,3,0.780855
7,avg1d,2,32,128,1,0.216055
8,conv1d,2,128,128,2,2.864581
9,max1d,2,128,256,1,0.872667


In [22]:
load('datasets/total_{}.{}'.format(2, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,avg1d,4,128,512,3,0.743448
1,avg1d,4,64,64,3,0.602544
2,conv1d,2,64,64,2,2.063026
3,dense,64,16,64,1,0.233751
4,norm2d,0,32,64,2,2.303360
5,max1d,2,32,64,1,0.081491
6,dense,32,128,64,1,0.944685
7,avg2d,4,32,512,3,9.380125
8,norm2d,0,64,256,1,21.593909
9,max2d,2,32,256,2,3.903939


In [27]:
load('datasets/total_{}.{}'.format(3, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,conv2d,4,128,32,2,20.121450
1,norm2d,0,32,256,2,7.704335
2,conv1d,8,32,64,3,1.869319
3,avg1d,2,128,64,1,0.107952
4,conv2d,4,64,32,1,4.598758
5,max2d,4,16,32,3,0.539124
6,conv1d,2,64,128,1,1.518969
7,conv2d,8,16,256,1,3.856755
8,max2d,2,64,32,1,1.469158
9,dense,128,16,512,2,0.687584


In [23]:
store_big(num=1,frame='torch', the_col='CPU time avg (us)')

In [24]:
store_big(num=2,frame='torch', the_col='CPU time avg (us)')

In [25]:
store_big(num=3,frame='torch', the_col='CPU time avg (us)')

In [28]:
load('datasets/big_total_{}.{}'.format(1, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,max1d,4,32,64,2,1.991587
1,max1d,4,64,64,1,0.792841
2,conv1d,4,128,128,3,8.347496
3,conv2d,2,64,32,1,8.968720
4,avg2d,2,64,32,3,3.093936
5,conv1d,8,128,512,3,13.453068
6,norm1d,0,64,128,3,2.653368
7,avg1d,2,32,128,1,0.964359
8,conv1d,2,128,128,2,9.263613
9,max1d,2,128,256,1,4.191648


In [26]:
load('datasets/big_total_{}.{}'.format(2, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,avg1d,4,128,512,3,3.509127
1,avg1d,4,64,64,3,1.842229
2,conv1d,2,64,64,2,6.771758
3,dense,64,16,64,1,0.790223
4,norm2d,0,32,64,2,5.458044
5,max1d,2,32,64,1,0.618311
6,dense,32,128,64,1,2.143695
7,avg2d,4,32,512,3,15.157231
8,norm2d,0,64,256,1,43.123364
9,max2d,2,32,256,2,10.651364


In [29]:
load('datasets/big_total_{}.{}'.format(3, 'torch'))

,model_str,hp,numf,batch,nodes,total_ms
0,conv2d,4,128,32,2,52.969128
1,norm2d,0,32,256,2,16.655733
2,conv1d,8,32,64,3,5.978349
3,avg1d,2,128,64,1,0.951242
4,conv2d,4,64,32,1,12.703704
5,max2d,4,16,32,3,1.625251
6,conv1d,2,64,128,1,4.937348
7,conv2d,8,16,256,1,12.430289
8,max2d,2,64,32,1,3.858423
9,dense,128,16,512,2,3.005207
